#### What is SCD Type-2 ?

- SCD Type-2 tracks changes in a dimension table by creating a new record each time a change occurs. 
- This allows you to maintain historical data. Each record is uniquely identified by a surrogate key and includes metadata such as:
1. Effective start and end dates: These indicate when a record was active.
2. Active flag: This indicates whether the record is the current one.
3. Version number: This tracks the number of changes to the record.

### Steps to Implement SCD Type-2 in PySpark

1. Create Initial Data (Source Table): Represent the incoming data with updates.
2. Create Dimension Table: Represent the target dimension table where data is stored.
3. Compare Source and Dimension Tables: Identify new, changed, and unchanged records.
4. Insert Updated and New Records: Update the dimension table accordingly.

### Import Library

In [50]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, current_date,expr
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

### Create Spark Session

In [51]:
spark = SparkSession\
    .builder\
    .master("local")\
    .appName("SCD Type-2 Example")\
    .getOrCreate()

### Create Source Data

In [52]:
source_df = spark\
    .read\
    .format("csv")\
    .option("header",True)\
    .option("inferSchema",True)\
    .load("/Users/sahilnagpal/Desktop/wordsToSpeak/SlowlyChangingDimension/source_file.csv")

source_df.show()

+---+-------+----------+--------------+
| id|   name|department|effective_date|
+---+-------+----------+--------------+
|  1|  Alice|        HR|    2024-01-01|
|  2|    Bob|   Finance|    2024-01-01|
|  3|Charlie|        IT|    2024-01-01|
|  4|  Diana|        HR|    2024-01-01|
|  5|    Eve|        IT|    2024-01-01|
+---+-------+----------+--------------+



### Create Dimension Data (Existing Dimension Table)

In [53]:
dimension_df = spark\
    .read\
    .format("csv")\
    .option("header",True)\
    .option("inferSchema",True)\
    .load("/Users/sahilnagpal/Desktop/wordsToSpeak/SlowlyChangingDimension/dimension_file.csv")

dimension_df.show()

+---+-------+----------+--------------------+------------------+---------+-------+
| id|   name|department|effective_start_date|effective_end_date|is_active|version|
+---+-------+----------+--------------------+------------------+---------+-------+
|  1|  Alice|        HR|          2024-01-01|        2099-12-31|     true|    1.0|
|  2|    Bob|   Finance|          2024-01-01|        2099-12-31|     true|    1.0|
|  3|Charlie|        IT|          2024-01-01|        2099-12-31|     true|    1.0|
|  4|  Diana| Marketing|          2024-01-01|        2099-12-31|     true|    1.0|
+---+-------+----------+--------------------+------------------+---------+-------+



### Compare Source and Dimension Tables

#### Implemented a join (specifically left join) based on matching columns which is ID here
##### Question Why Left Join Only : 

In [54]:
# Join source and dimension data on 'id' to compare
comparison_df = source_df.join(
    dimension_df,
    on="id",
    how="left_outer"
)

In [55]:
comparison_df.show()

+---+-------+----------+--------------+-------+----------+--------------------+------------------+---------+-------+
| id|   name|department|effective_date|   name|department|effective_start_date|effective_end_date|is_active|version|
+---+-------+----------+--------------+-------+----------+--------------------+------------------+---------+-------+
|  1|  Alice|        HR|    2024-01-01|  Alice|        HR|          2024-01-01|        2099-12-31|     true|    1.0|
|  2|    Bob|   Finance|    2024-01-01|    Bob|   Finance|          2024-01-01|        2099-12-31|     true|    1.0|
|  3|Charlie|        IT|    2024-01-01|Charlie|        IT|          2024-01-01|        2099-12-31|     true|    1.0|
|  4|  Diana|        HR|    2024-01-01|  Diana| Marketing|          2024-01-01|        2099-12-31|     true|    1.0|
|  5|    Eve|        IT|    2024-01-01|   NULL|      NULL|                NULL|              NULL|     NULL|   NULL|
+---+-------+----------+--------------+-------+----------+------

### a) Identify New Records (Not in Dimension Table)

- In this case we first check for new records, this is done by checking if matching column in existing data is missing (or null).
- In the example below, we know ```id=5``` is missing in dimension table and coming new from source

In [56]:
new_records = comparison_df\
    .filter(dimension_df.id.isNull())\
    .select(
    source_df["*"],
    lit("2099-12-31").cast(DateType()).alias("effective_end_date"),
    lit(True).alias("is_active"),
    lit(1).alias("version")
)

new_records.show()

+---+----+----------+--------------+------------------+---------+-------+
| id|name|department|effective_date|effective_end_date|is_active|version|
+---+----+----------+--------------+------------------+---------+-------+
|  5| Eve|        IT|    2024-01-01|        2099-12-31|     true|      1|
+---+----+----------+--------------+------------------+---------+-------+



### b) Identify Changed Records (Values are Different)
- In this case , we will check for non-matching records where we essentially put two conditions which are as follows:
1. Dimension ID is not null
2. Other Key columns are different in source and existing dimension table

- In our case for ```id=4``` the record has same ```name``` but different ```department name``` so it will appear in change dataframe.

In [57]:
changed_records = comparison_df\
    .filter(
    (dimension_df.id.isNotNull()) & 
    ((source_df.name != dimension_df.name) | (source_df.department != dimension_df.department))
    )


changed_records.show()

+---+-----+----------+--------------+-----+----------+--------------------+------------------+---------+-------+
| id| name|department|effective_date| name|department|effective_start_date|effective_end_date|is_active|version|
+---+-----+----------+--------------+-----+----------+--------------------+------------------+---------+-------+
|  4|Diana|        HR|    2024-01-01|Diana| Marketing|          2024-01-01|        2099-12-31|     true|    1.0|
+---+-----+----------+--------------+-----+----------+--------------------+------------------+---------+-------+



### c) Get Updated Records
- In this case, we are going to get source information since this is going to be our latest info.
- For this we will add all the columns matching with exisiting dimension table and following steps will happen:
1. The source date column will become our ```effective_start_date```.
2. Mark ```is_active``` flag as ```TRUE```.
3. Increase the counter of version from dimension table by 1.

In [58]:
updated_records = changed_records\
    .select(source_df["id"],
            source_df["name"],
            source_df["department"],
            source_df["effective_date"].alias("effective_start_date"),
            lit("2099-12-31").cast(DateType()).alias("effective_end_date"),
            lit(True).alias("is_active"),
            (dimension_df.version + 1).alias("version")
            )

updated_records.show()

+---+-----+----------+--------------------+------------------+---------+-------+
| id| name|department|effective_start_date|effective_end_date|is_active|version|
+---+-----+----------+--------------------+------------------+---------+-------+
|  4|Diana|        HR|          2024-01-01|        2099-12-31|     true|    2.0|
+---+-----+----------+--------------------+------------------+---------+-------+



### d) Expire old records in the dimension table

In [59]:
expired_records = changed_records.select(
    dimension_df["*"]
).withColumn("is_active", lit(False))

expired_records.show()

+---+-----+----------+--------------------+------------------+---------+-------+
| id| name|department|effective_start_date|effective_end_date|is_active|version|
+---+-----+----------+--------------------+------------------+---------+-------+
|  4|Diana| Marketing|          2024-01-01|        2099-12-31|    false|    1.0|
+---+-----+----------+--------------------+------------------+---------+-------+



### Union All Updates and New Data and Combine expired records with updates to form the final dimension table

In [60]:
# Union All Updates and New Data
scd2_updates = updated_records.union(new_records)

scd2_updates.show()

+---+-----+----------+--------------------+------------------+---------+-------+
| id| name|department|effective_start_date|effective_end_date|is_active|version|
+---+-----+----------+--------------------+------------------+---------+-------+
|  4|Diana|        HR|          2024-01-01|        2099-12-31|     true|    2.0|
|  5|  Eve|        IT|          2024-01-01|        2099-12-31|     true|    1.0|
+---+-----+----------+--------------------+------------------+---------+-------+



In [61]:
# Combine expired records with updates to form the final dimension table

final_df = dimension_df\
    .union(scd2_updates)\
    .union(expired_records)

final_df.show()

+---+-------+----------+--------------------+------------------+---------+-------+
| id|   name|department|effective_start_date|effective_end_date|is_active|version|
+---+-------+----------+--------------------+------------------+---------+-------+
|  1|  Alice|        HR|          2024-01-01|        2099-12-31|     true|    1.0|
|  2|    Bob|   Finance|          2024-01-01|        2099-12-31|     true|    1.0|
|  3|Charlie|        IT|          2024-01-01|        2099-12-31|     true|    1.0|
|  4|  Diana| Marketing|          2024-01-01|        2099-12-31|     true|    1.0|
|  4|  Diana|        HR|          2024-01-01|        2099-12-31|     true|    2.0|
|  5|    Eve|        IT|          2024-01-01|        2099-12-31|     true|    1.0|
|  4|  Diana| Marketing|          2024-01-01|        2099-12-31|    false|    1.0|
+---+-------+----------+--------------------+------------------+---------+-------+

